In [1]:
import datetime
import pickle

import pandas as pd
from river import metrics, optim, reco

In [2]:
with open("../data/model.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
df = pd.read_pickle("../data/trainset.pkl")
anime_list = df[["Name", "Japanese_name", "anime_id", "Score"]].drop_duplicates()

In [4]:
now = datetime.datetime.now()

In [5]:
user_id = now.strftime("%Y%m%d%H%M%S")
user_id = int(user_id)
print(user_id)

20230623171424


In [6]:
test_data = [
    "Angel Beats!",
    "Mahoutsukai Precure!",
    "Ookami to Koushinryou",
    "Shinsekai yori",
    "Shugo Chara!",
    "Uma Musume: Pretty Derby (TV)",
    "Fate/Zero",
    "Uchuu Senkan Yamato 2199",
]

In [7]:
test_df = anime_list[anime_list["Name"].isin(test_data)]
test_df["user_id"] = user_id
test_df["rating"] = 10

/tmp/ipykernel_474199/2289128984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["user_id"] = user_id
/tmp/ipykernel_474199/2289128984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["rating"] = 10


In [8]:
test_df.head()

,Name,Japanese_name,anime_id,Score,user_id,rating
1527811,Angel Beats!,Angel Beats!（エンジェルビーツ）,6547,8.15,20230623171424,10
12121575,Fate/Zero,フェイト/ゼロ,10087,8.34,20230623171424,10
20546661,Ookami to Koushinryou,狼と香辛料,2966,8.27,20230623171424,10
27276805,Shinsekai yori,新世界より,13125,8.35,20230623171424,10
32041714,Shugo Chara!,しゅごキャラ！,2923,7.43,20230623171424,10


In [9]:
testset = test_df[["user_id", "anime_id", "rating"]].to_dict(orient="records")

In [10]:
metric = metrics.MAE() + metrics.RMSE()

In [11]:
# Update model
for data in testset:
    y_pred = model.predict_one(user=data["user_id"], item=data["anime_id"])
    metric = metric.update(data["rating"], y_pred)
    model = model.learn_one(
        user=data["user_id"], item=data["anime_id"], y=data["rating"]
    )

In [12]:
metric

MAE: 1.790617, RMSE: 1.822808

In [13]:
with open("../data/retrain_model.pkl", "wb") as f:
    pickle.dump(model, f)

In [14]:
def predict(uid, df, anime_list):
    # predict recommend item
    result_df = model.rank(user=uid, items=anime_list["anime_id"])
    result_df = pd.DataFrame(result_df, columns=["anime_id"])
    result_df = pd.merge(result_df, anime_list, on=["anime_id"], how="inner")

    # remove rated item
    rated_df = df[df["user_id"] == uid]["anime_id"]
    result_df = result_df[~result_df["anime_id"].isin(rated_df)]
    result_df = result_df.reset_index(drop=True)[0:10]

    return result_df

In [15]:
predict(user_id, df, anime_list)

,anime_id,Name,Japanese_name,Score
0,457,Mushishi,蟲師,8.69
1,9253,Steins;Gate,STEINS;GATE,9.11
2,2004,Hanada Shounen-shi,花田少年史,8.01
3,21939,Mushishi Zoku Shou,蟲師 続章,8.72
4,820,Ginga Eiyuu Densetsu,銀河英雄伝説,9.07
5,6594,Katanagatari,刀語,8.36
6,5941,Cross Game,クロスゲーム,8.43
7,24701,Mushishi Zoku Shou 2nd Season,蟲師 続章,8.76
8,10087,Fate/Zero,フェイト/ゼロ,8.34
9,3297,Aria the Origination,ARIA The ORIGINATION,8.53
